In [1]:
import sys, os
import tensorflow as tf
sys.path.append("..")
import tools
import evals
from astroML.crossmatch import crossmatch_angular
import numpy as np
import pandas as pd
import multiprocessing
from lsst.daf.butler import Butler
import time
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2

In [64]:
#model_path = "../DATA/Trained_model_0.keras"
model_path = "../DATA/trained1.keras"
tf_dataset_path = "/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/"
collection = "u/kmrakovc/"
val_index_path="/astro/users/kmrakovc/Projects/LSST_Streak_Detection/DATA/val_index1.npy"
repo_path ="/epyc/ssd/users/kmrakovc/DATA/rc2_subset/SMALL_HSC/"
threshold = 0.1
batch_size=10

In [78]:
def one_LSST_stack_comparison(butler, output_coll, injection_catalog_id, source_catalog_id, calexp_id, calexp_dimensions,
                              column_name):
    injection_catalog = butler.get("injected_postISRCCD_catalog",
                                   dataId=injection_catalog_id.dataId,
                                   collections=output_coll, )
    original_source_catalog = butler.get("src",
                                         dataId=source_catalog_id.dataId,
                                         collections=output_coll, )
    source_catalog = butler.get("injected_src",
                                dataId=source_catalog_id.dataId,
                                collections=output_coll, )
    calexp = butler.get("injected_calexp.wcs",
                        dataId=calexp_id.dataId,
                        collections=output_coll)
    sc = source_catalog.asAstropy().to_pandas()
    osc = original_source_catalog.asAstropy().to_pandas()
    dist, ind = crossmatch_angular(sc[['coord_ra', 'coord_dec']].values,
                                   osc[['coord_ra', 'coord_dec']].values, 0.04 / 3600)
    source_origin = calexp.skyToPixelArray(np.array([source_catalog["coord_ra"][np.isinf(dist)]]),
                                                    np.array([source_catalog["coord_dec"][np.isinf(dist)]]),
                                                    degrees=False)
    injected_origin = calexp.skyToPixelArray(np.array([injection_catalog["ra"]]),
                                                      np.array([injection_catalog["dec"]]),
                                                      degrees=True)
    angle = injection_catalog["beta"]
    length = injection_catalog["trail_length"]
    mask_source = np.zeros(calexp_dimensions)
    mask_source[source_origin[1].astype(int), source_origin[0].astype(int)] = 1
    matched_values = np.array([])
    for j in range(len(angle)):
        mask_inject = tools.data.draw_one_line(np.zeros(calexp.image.array.shape),
                                               (injected_origin[0][j], injected_origin[1][j]),
                                               angle[j], length[j])
        if (mask_inject * mask_source).sum() > 0:
            matched_values = np.append(matched_values, injection_catalog[column_name][j])
    return matched_values


def LSST_stack_comparation_histogram_data(repo, output_coll, val_index_path,
                                          column_name="trail_length", batch_size=None):
    from lsst.daf.butler import Butler
    with open(val_index_path, 'rb') as f:
        val_index = np.load(f)
        val_index.sort()
    butler = Butler(repo)
    injection_catalog_ids = list(
        butler.registry.queryDatasets("injected_postISRCCD_catalog", collections=output_coll, instrument='HSC'))
    source_catalog_ids = list(butler.registry.queryDatasets("injected_src", collections=output_coll, instrument='HSC'))
    calexp_ids = list(butler.registry.queryDatasets("injected_calexp", collections=output_coll, instrument='HSC'))
    calexp_dimensions = butler.get("injected_calexp.dimensions", dataId=ref[0].dataId, collections=output_coll)
    calexp_dimensions = (calexp_dimensions.y, calexp_dimensions.x)
    parameters = [(butler, output_coll,
                   injection_catalog_ids[i], source_catalog_ids[i],
                   calexp_ids[i], calexp_dimensions, column_name) for i in val_index]
    if batch_size is None:
        batch_size = max(1, os.cpu_count() - 1)
    if batch_size>1:
        with multiprocessing.Pool(batch_size) as pool:
            list_cat = pool.starmap(one_LSST_stack_comparison, parameters)
    else:
        list_cat = [None] * len(parameters)
        for i, p in enumerate(parameters):
            list_cat[i] = one_LSST_stack_comparison(*p)
    return np.concatenate(list_cat)

In [79]:
start_time = time.time()
LSST_stack_detected_asteroids_1m= evals.eval_tools.LSST_stack_comparation_histogram_data(repo_path,
                                                                                        collection+"single_frame_injection_01",
                                                                                        val_index_path,
                                                                                        batch_size=10,
                                                                                        column_name="mag")
print (time.time()-start_time)

In [80]:
start_time = time.time()
LSST_stack_detected_asteroids_1m= evals.eval_tools.LSST_stack_comparation_histogram_data(repo_path,
                                                                                        collection+"single_frame_injection_01",
                                                                                        val_index_path,
                                                                                        batch_size=1,
                                                                                        column_name="mag")
print (time.time()-start_time)